# PPTX 도형별 텍스트 및 폰트 정보 추출

이 노트북은 PPTX 파일에서 각 도형의 텍스트와 글씨체(폰트) 정보를 추출합니다.

In [ ]:
# 필요한 라이브러리 설치 (처음 한 번만 실행)
# !pip install python-pptx
# "•", "◦", "▪", "▶", "→" "➔"

In [19]:
from pptx import Presentation
from pptx.util import Pt

In [20]:
# 분석할 PPTX 파일 경로 설정
pptx_path = "wr_base.pptx"  # 여기에 파일 경로를 입력하세요

In [21]:
def extract_shape_info(pptx_path):
    """PPTX 파일에서 각 도형의 텍스트와 폰트 정보 추출"""
    prs = Presentation(pptx_path)
    results = []
    
    for slide_idx, slide in enumerate(prs.slides, 1):
        slide_info = {
            "슬라이드": slide_idx,
            "도형들": []
        }
        
        for shape_idx, shape in enumerate(slide.shapes, 1):
            shape_info = {
                "도형번호": shape_idx,
                "도형이름": shape.name,
                "도형타입": shape.shape_type.name if hasattr(shape.shape_type, 'name') else str(shape.shape_type),
                "텍스트": [],
                "폰트정보": []
            }
            
            # 텍스트 프레임이 있는 도형만 처리
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    para_text = paragraph.text
                    if para_text.strip():
                        shape_info["텍스트"].append(para_text)
                    
                    # 각 run의 폰트 정보 추출
                    for run in paragraph.runs:
                        if run.text.strip():
                            font = run.font
                            font_info = {
                                "텍스트": run.text,
                                "폰트명": font.name if font.name else "(기본값)",
                                "크기": f"{font.size.pt}pt" if font.size else "(기본값)",
                                "굵게": font.bold if font.bold is not None else "(상속)",
                                "기울임": font.italic if font.italic is not None else "(상속)",
                                "밑줄": font.underline if font.underline is not None else "(상속)",
                            }
                            shape_info["폰트정보"].append(font_info)
            
            # 텍스트가 있는 도형만 결과에 추가
            if shape_info["텍스트"] or shape_info["폰트정보"]:
                slide_info["도형들"].append(shape_info)
        
        if slide_info["도형들"]:
            results.append(slide_info)
    
    return results

In [22]:
def print_results(results):
    """결과를 보기 좋게 출력"""
    print("=" * 80)
    print("PPTX 도형별 텍스트 및 폰트 정보")
    print("=" * 80)
    
    for slide in results:
        print(f"\n📄 슬라이드 {slide['슬라이드']}")
        print("-" * 60)
        
        for shape in slide["도형들"]:
            print(f"\n  🔷 도형 {shape['도형번호']}: {shape['도형이름']}")
            print(f"     타입: {shape['도형타입']}")
            
            if shape["텍스트"]:
                print(f"     📝 텍스트:")
                for text in shape["텍스트"]:
                    print(f"        \"{text}\"")
            
            if shape["폰트정보"]:
                print(f"     🔤 폰트 정보:")
                for font in shape["폰트정보"]:
                    text_preview = font['텍스트'][:30] + '...' if len(font['텍스트']) > 30 else font['텍스트']
                    print(f"        - \"{text_preview}\"")
                    print(f"          폰트: {font['폰트명']}, 크기: {font['크기']}")
                    print(f"          굵게: {font['굵게']}, 기울임: {font['기울임']}")

In [23]:
# 실행
results = extract_shape_info(pptx_path)
print_results(results)

PPTX 도형별 텍스트 및 폰트 정보

📄 슬라이드 1
------------------------------------------------------------

  🔷 도형 2: TextBox 21
     타입: TEXT_BOX
     📝 텍스트:
        "AI Weekly Report"
     🔤 폰트 정보:
        - "AI Weekly Report"
          폰트: 한화 R, 크기: 32.0pt
          굵게: (상속), 기울임: (상속)

  🔷 도형 5: TextBox 1
     타입: TEXT_BOX
     📝 텍스트:
        "제 13호 | 2025년 12월 12일"
     🔤 폰트 정보:
        - "제 "
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "13"
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "호 "
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "|"
          폰트: 한화고딕 R, 크기: 11.0pt
          굵게: True, 기울임: (상속)
        - " 2025"
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "년 "
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "12"
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
        - "월 "
          폰트: 한화고딕 L, 크기: 11.0pt
          굵게: (상속), 기울임: (상속)
  

In [24]:
# (선택) 결과를 도형별로 정렬하여 보기
import pandas as pd

# 도형별로 텍스트와 폰트 정보를 그룹화
grouped_data = []
for slide in results:
    for shape in slide["도형들"]:
        # 도형의 모든 텍스트를 수집
        all_texts = " ".join(shape["텍스트"]) if shape["텍스트"] else ""
        
        # 도형에서 사용된 폰트들을 수집 (중복 제거)
        fonts_used = set()
        for font in shape["폰트정보"]:
            fonts_used.add(font["폰트명"])
        
        grouped_data.append({
            "슬라이드": slide["슬라이드"],
            "도형이름": shape["도형이름"],
            "도형타입": shape["도형타입"],
            "텍스트": all_texts,
            "사용된폰트": ", ".join(sorted(fonts_used))
        })

df_grouped = pd.DataFrame(grouped_data)
print("\n📊 도형별 텍스트 및 폰트 정보:\n")
print(df_grouped.to_string(index=False))
print("\n" + "="*100)
print("\n자세히 보기 (DataFrame):\n")
df_grouped


📊 도형별 텍스트 및 폰트 정보:

 슬라이드                               도형이름       도형타입                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      텍스트           사용된폰트
    1                         TextBox 21   TEXT_BOX                                                                   

,슬라이드,도형이름,도형타입,텍스트,사용된폰트
0,1,TextBox 21,TEXT_BOX,AI Weekly Report,한화 R
1,1,TextBox 1,TEXT_BOX,제 13호 | 2025년 12월 12일,"한화고딕 L, 한화고딕 R"
2,1,TextBox 2,TEXT_BOX,"신한라이프, 생성형 AI를 활용한 ‘상담 요약 안내 서비스‘ 도입 상담 통화 내용을...","한화고딕 B, 한화고딕 EL"
3,1,Google Shape;202;g375bbbb1663_0_21,AUTO_SHAPE,국내외 AI 동향,한화 R
4,1,Google Shape;202;g375bbbb1663_0_21,AUTO_SHAPE,AI Lab,한화 R
5,1,TextBox 9,TEXT_BOX,DT(Digital Transformation) 워킹그룹 추진 DT 워킹그룹 참여자...,"한화고딕 B, 한화고딕 EL"
6,1,Google Shape;202;g375bbbb1663_0_21,AUTO_SHAPE,HAC,한화 R


In [1]:
summarized_text='''
[Title] 미래에셋생명, AI 기반 맞춤형 건광관리 서비스 ‘헬스케어 AI’ 제공 [Summary1] 사용자의 건강검진 기록과 의료 데이터를 종합해 주요 질환의 발병 가능성을 예측하여 , 사용자가 선제적으로 위험 인자를 관리할 수 있도록 도움 제공 [Summary2] 미래에셋생명의 AI 헬스케어 서비스는 질병 예측 AI, 기대 수명 예측, 의료비 예측, 개인 맞춤 건강 가이드 네 가지 핵심 특징을 갖추고 있음 [Insight] 당사도 향후 여성 헬스케어 서비스를 제공할 경우, 여성에게서 빈번하게 발생하는 주요 질환의 발병 ….가능성과 예상 의료비 기반으로 개인별 건강·재무 계획 수립을 지원할 수 있을것으로 기대됨 [Title] 카카오뱅크, 생성형 AI를 활용해 대화만으로 송금할 수 있는 ‘AI이체‘ 출시 [Summary1] 최근 이체 내역에 대상의 이름이 있는 경우, 대화창에 ‘김이지에게 오만원 보내줘＇와 같이 입력하면 은행명이나 계좌번호를 직접 입력하지 않아도 대상의 이름만으로 이체 가능 [Summary2] 고객 요청이 불분명하면 재질문을 통해 질문을 명확히 하거나 수취인 검증 등 일반 이체와 동일한 인증 절차를 거치게 하는 등 착오송금 방지를 위한 절차도 강화  [Insight] 당사 AI Lab에서도 AI 챗봇으로 온라인 채널 내 서비스/기능을 간편하게 연결할 예정이며, 향후 기술 ….발전에 맞추어 고객 편의 향상을 위한 AI 기반 기능을 지속 확대할 계획임 [Title] 구글, 성능이 크게 향상된 차세대 AI 모델 ‘제미나이 3’ 공개 [Summary1] 고난도 추론이 가능한 ‘제미나이3 딥싱크 (Gemini 3 Deep Think)’ 모델, 에이전트 중심 개발 플랫폼인 ‘구글 안티그래비티 (Google Antigravity)’도 공개 [Summary2] 모델의 사전 훈련과 사후 훈련이 개선되어 성능이 향상된 것으로 분석됨 [Insight] 제미나이 3 및 계속해서 고도화되고 있는 생성형 AI 활용으로 직원들의 업무 효율성 향상 기대
'''

In [ ]:
import re
from pptx import Presentation
from pptx.util import Pt

TAG_RE = re.compile(r'\[(Title|Summary1|Summary2|Insight)\]\s*', re.IGNORECASE)

# 태그별 스타일 설정 (prefix, font_name, font_size, underline, split_lines)
TAG_STYLES = {
    "title":    ("",   "한화고딕 B",  12, False, False),
    "summary1": ("• ", "한화고딕 EL", 12, False, True),
    "summary2": ("• ", "한화고딕 EL", 12, False, True),
    "insight":  ("➔ ", "한화고딕 B",  12, True,  True),
}
DEFAULT_STYLE = ("", "한화고딕 EL", 12, False, True)

def parse_sections(text: str):
    matches = list(TAG_RE.finditer(text))
    return [
        (m.group(1).lower(), text[m.end():matches[i+1].start() if i+1 < len(matches) else len(text)].strip())
        for i, m in enumerate(matches)
        if text[m.end():matches[i+1].start() if i+1 < len(matches) else len(text)].strip()
    ]

def find_shape_by_name(prs: Presentation, name: str):
    for slide in prs.slides:
        if shape := next((s for s in slide.shapes if getattr(s, "name", None) == name), None):
            return slide, shape
    return None, None

def add_styled_run(paragraph, text, font_name, font_size, underline=False):
    r = paragraph.add_run()
    r.text = text
    r.font.name = font_name
    r.font.size = Pt(font_size)
    r.font.underline = underline

def set_textbox2_from_summarizedtxt(pptx_in: str, pptx_out: str, text: str, shape_name: str = "TextBox 2"):
    prs = Presentation(pptx_in)
    _, shape = find_shape_by_name(prs, shape_name)
    
    if not shape:
        raise ValueError(f'"{shape_name}" shape을 찾지 못했습니다.')
    if not shape.has_text_frame:
        raise ValueError(f'"{shape_name}"에 text_frame이 없습니다.')

    tf = shape.text_frame
    tf.clear()
    sections = parse_sections(text)

    if not sections:
        add_styled_run(tf.paragraphs[0], text.strip(), "한화고딕 EL", 12)
        prs.save(pptx_out)
        return

    first_para_used = False
    for tag, content in sections:
        prefix, font_name, font_size, underline, split = TAG_STYLES.get(tag, DEFAULT_STYLE)
        lines = [ln.strip() for ln in content.splitlines() if ln.strip()] if split else [content.strip()]

        for line in filter(None, lines):
            p = tf.paragraphs[0] if not first_para_used and not tf.paragraphs[0].text else tf.add_paragraph()
            first_para_used = True
            add_styled_run(p, f"{prefix}{line}" if prefix else line, font_name, font_size, underline)

        if tag == "insight":
            add_styled_run(tf.add_paragraph(), " ", "한화고딕 EL", 9)

    prs.save(pptx_out)